<a href="https://colab.research.google.com/github/nandhukumar86/CapstonePneumoniaDetection/blob/main/Vani/Capstoneyolomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
TrainDir = "/content/drive/MyDrive/Content/DriveAIML/capstone/jpg_reshaped_train/"
TestDir='/content/drive/MyDrive/Content/DriveAIML/capstone/jpg_reshaped_test/'
basepath = "/content/drive/MyDrive/Content/DriveAIML/capstone/"

In [3]:
import os, cv2,copy, gc
import pickle
import glob2 as glob
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, Model, losses
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation, BatchNormalization

In [58]:
!pip install pydicom

     |████████████████████████████████| 1.9MB 4.3MB/s 


In [4]:
filename =  '/content/drive/MyDrive/Content/DriveAIML/capstone/RezisedLabel.pickle'
with open(filename, "rb") as input_file:
  y_train = pickle.load(input_file)

In [5]:
trainimg = glob.glob(TrainDir+'*.jpg') #Getting all images in this folder
testimg=glob.glob(TestDir+"*.jpg") #

In [6]:
 len(trainimg)

11740

In [7]:
y_train.head(5)


,patientId,x,y,width,height,Target,X,Y,Xo,Yo,xo,yo,widtho,heighto
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,1024,1024,256,256,66.0,38.0,54.0,95.0


In [8]:
y_train.size

423178

In [9]:
class_info_df = pd.read_csv(basepath+'stage_2_detailed_class_info.csv')

In [10]:
## Merging the two datasets
#In class detailed info dataset are given the detailed information about 
#the type of positive or negative class associated with a certain patient.
#In y_train labels dataset are given the patient ID and the window (x min, y min, width and height of the) 
#containing evidence of pneumonia
train_class_df = y_train.merge(class_info_df, left_on= ('patientId'), right_on=('patientId'), how='inner').drop_duplicates()
train_class_df.sample(5)

,patientId,x,y,width,height,Target,X,Y,Xo,Yo,xo,yo,widtho,heighto,class
29856,d4d10646-e095-4cde-9f9a-b0d4b4cf6d5c,564.0,177.0,253.0,503.0,1,1024,1024,256,256,141.0,45.0,64.0,126.0,Lung Opacity
10239,5a6172a3-2b11-4e63-a8d5-f06829735f32,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal
4183,354c3756-43ed-4921-adf8-60be49a8b7e8,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal
14631,78cc9897-ddd1-4e87-8ce4-c05192891e76,517.0,592.0,192.0,142.0,1,1024,1024,256,256,130.0,148.0,48.0,36.0,Lung Opacity
23528,b0fd45a3-14f6-4462-8dd4-1d643bb517d4,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal


In [14]:
#reading image samples
sampNum = 500 # Number of samples to import
X = [] # Empty to read training images
x1=[]
print(type(x1))
print("Before sampling\n")
print("Total training unique patient ids: ",train_class_df['patientId'].nunique())
print("Number of images in train set:", len(os.listdir(TrainDir)))
print("\n")
print("Total training labels {}" .format(train_class_df.shape))
#display(distTab(train_class_df, "Target")) # Dsitribution of target before sampling

print("\n\nAfter sampling\n")
Y = train_class_df.sample(n=sampNum, random_state=2) # Sampling
Y.reset_index(drop=True, inplace=True)
print("Total sampled training labels {}" .format(Y.shape))
#display(distTab(Y.sample(n=sampNum), "Target"))  # Dsitribution of target after sampling

for indx in Y.patientId:
    
    try:
       #print(indx)
       filepath=TrainDir+indx+'.jpg'
       image = cv2.imread(filepath)
       X.append(indx)
       x1.append(image)
    except: 
        continue
zipbObj = zip(X, x1)        
 

X=dict(zipbObj)
#print("df chk",Y[Y.patientId.isin(X.keys())])
#X = dict(zip([x.patientId for x in X], [x.pixel_array for x in X])) # Dictionary for training images for patient id's
print("\n\nTotal sampled training images {}" .format(len(X)))
Y = Y[Y.patientId.isin(X.keys())]
print("Total training labels {}" .format(Y.shape))

<class 'list'>
Before sampling

Total training unique patient ids:  26684
Number of images in train set: 11740


Total training labels (30227, 15)


After sampling

Total sampled training labels (500, 15)


Total sampled training images 500
Total training labels (500, 15)


In [15]:
# Shape of images
len(X)

500

In [17]:
Xdf = pd.DataFrame({'patientId':X.keys(),
                   'imageArray':X.values()})
dat = Y.merge(Xdf, on='patientId', how='left')

In [18]:
dat.head(5)

,patientId,x,y,width,height,Target,X,Y,Xo,Yo,xo,yo,widtho,heighto,class,imageArray
0,e4dd25aa-7839-4e0e-a984-c7cb1b31d2c7,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal,"[[[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], ..."
1,5c389c6d-2abc-43a9-91e7-847f85bf6ff1,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None
2,324f9ed1-09e0-483e-9bdd-d139e77acb85,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal,None
3,4ac6590a-ecdb-47b7-92f3-cfc6fa405955,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None
4,7213e146-4efd-40e6-ab88-a9e7a9c8c25c,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None


In [19]:
Xdf = pd.DataFrame({'patientId':X.keys(), 'imageArray':X.values()})
dat = Y.merge(Xdf, on='patientId', how='left')
dat.head()

,patientId,x,y,width,height,Target,X,Y,Xo,Yo,xo,yo,widtho,heighto,class,imageArray
0,e4dd25aa-7839-4e0e-a984-c7cb1b31d2c7,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal,"[[[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], ..."
1,5c389c6d-2abc-43a9-91e7-847f85bf6ff1,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None
2,324f9ed1-09e0-483e-9bdd-d139e77acb85,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal,None
3,4ac6590a-ecdb-47b7-92f3-cfc6fa405955,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None
4,7213e146-4efd-40e6-ab88-a9e7a9c8c25c,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None


In [154]:
Xdf = pd.DataFrame({'patientId':X.keys(),
                   'imageArray':X.values()})

dat = Y.merge(Xdf, on='patientId', how='left')

